# **MATH&ML-15. Рекомендательные системы. Часть II**

# 1. Введение

✍ В предыдущем модуле мы начали знакомиться с рекомендательными системами — кратко разобрали существующие подходы к их построению и научились оценивать качество РС. Более подробно мы пока успели изучить только popularity-based подход. В этом модуле мы рассмотрим остальные алгоритмы и научимся реализовывать их на практике.

Для начала давайте кратко повторим материал, изученный в прошлом модуле ↓

Отлично! Теперь вы точно готовы к дальнейшему изучению рекомендательных систем.

Основные цели, которые стоят перед нами в этом модуле:

1. Разобрать принципы работы моделей рекомендательных систем:
    * content-based-модели,
    * коллаборативной фильтрации,
    * гибридной модели.
2. Познакомиться с основами применения глубокого обучения для построения рекомендательных систем.
3. Отработать изученные алгоритмы на решении практических задач.

В [юните «Практика»](https://lms.skillfactory.ru/courses/course-v1:SkillFactory+DST-3.0+28FEB2021/jump_to_id/9af88f62b92b44559b442bfcc9f4d23d) мы достроим начатую в предыдущем модуле модель рекомендательной системы для статей, а в промежуточных юнитах рассмотрим применение методов построения рекомендательных систем на других датасетах из различных областей.

# 2. Content-based model

✍ В этом юните мы рассмотрим систему рекомендаций на основе контента, или, как её чаще называют, **content-based model**.

![](https://img.genial.ly/5fdc5ca1853b5759f6e69400/392c8cc2-df07-48b3-88f4-088149cb53fe.png)

> Подход **content-based** предполагает, что пользователю рекомендуются товары или контент на основе его предпочтений и вкусов.

Профиль интересов пользователя формируется исходя из его оценок, а также неявной обратной связи: лайков, количества просмотров и так далее. В качестве рекомендаций пользователю предлагаются похожие элементы.

Сходство или близость элементов измеряется на основе сходства содержания этих элементов. Говоря «содержание», мы имеем в виду такие сущности, как категория, тег, жанр и т. д., то есть метаданные. Далее в этом юните мы познакомимся с некоторыми алгоритмами определения сходства.

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/792c9ad6bbaf1201d6cfecf6d816838f/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/DST_MATH_ML_15_2_1.png)

Вы наверняка уже сталкивались с content-based-рекомендациями в тех или иных сервисах. Например, на сайте Netflix можно использовать фильтрацию по контенту, чтобы создавать рекомендации из аналогичных элементов, которые размещаются в разделе More Like This.

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/a8a0464a8baddb3f5b705ff2dc2f4945/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/DST_MATH_ML_15_2_2.png)

Давайте на примере рассмотрим построение рекомендательной системы на основе контента для конкретного пользователя.
Допустим, пользователь Михаил выставил лайки и дизлайки для фильмов на одном из веб-сервисов:

**ФИЛЬМ**|**РЕЙТИНГ**
-|-
«Миссия невыполнима»|Лайк
«Джеймс Бонд»|Лайк
«Приключения Буратино»|Дизлайк

Предположим, что по правилам сервиса лайк прибавляет 4.5 балла к фильмам с таким жанром, а дизлайк вычитает 6 баллов. Теперь создадим вектор пользователя для Михаила на основе трёх его оценок:

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/118b2ca3c07dba4356d56b26cb692aec/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/DST_MATH_ML_15_2_3.png)

Присваиваем значение 9 боевикам, так как Михаил поставил лайк двум фильмам с жанром «боевик». Михаил не смотрел анимационные фильмы, так что присваиваем 0 анимации, и, поскольку он оставил плохой отзыв фильму в жанре «дети», присваиваем -6 детским фильмам.

Таким образом, вектор пользователя для Михаила — это $(9,0,-6)$ для шкал (Боевик, Анимация, Дети).
Теперь попробуем предсказать отношение Михаила к фильмам, которые он ещё не смотрел, например «Звёздные войны» и «История игрушек».

«Звёздные войны» — это боевик, который не относится к анимации или детским фильмам, поэтому по шкалам (Боевик, Анимация, Дети) у этого фильма будут координаты $(1,0,0)$. У фильма «История игрушек» будут координаты $(0,1,1)$, так как он относится и к жанру детских фильмов, и к анимации.
Теперь нам необходимо найти произведения вектора пользователя и вектора фильма: чем больше будет скалярное произведение, тем более подходящим для Михаила будет фильм.

Скалярное произведение для «Истории игрушек» равно $-6$, а для «Звёздных войн» оно равно $9$. Следовательно, из этих двух вариантов именно «Звёздные войны» будут рекомендованы Михаилу, что вполне логично, ведь ему больше нравятся боевики.

Аналогично можно вычислить скалярные произведения векторов для всех фильмов на сайте и рекомендовать Михаилу десять наиболее подходящих фильмов.

В данном примере мы опирались лишь на схожесть фильмов по жанрам: скалярное произведение будет увеличиваться в том случае, если у фильма ненулевая координата для того жанра, который пользователь уже оценил положительно. Однако в реальности всё намного сложнее: например, мы можем использовать одновременно жанр фильма, теги, описание, актёров и т. д. Для этого случая нам нужен способ измерить сходство между набором параметров.

Рассмотрим два наиболее популярных метода измерения такого сходства:

* **индекс Жаккара**,
* **косинусная близость**.

### ИНДЕКС ЖАККАРА

Индекс Жаккара измеряет сходство между двумя наборами A и B как мощность множества пересечения, делённую на мощность множества объединения каких-то характеристик объекта. Его удобно применять для категориальных признаков.

$$J(A,B) = \frac{\left|A \cap B \right|}{\left|A \cup B \right|} = \frac{\left|A \cap B \right|}{\left|A \right| + \left|B \right| - \left|A \cap B \right|}$$

Например, с помощью индекса Жаккара мы можем оценить, насколько похожи фильмы, основываясь на наборах ключевых слов (тегов) для них:

* фильм А : {фантастика, школа, романтика};
* фильм B : {приключения, фантастика, школа};
* фильм C : {ужасы, триллер, драма}.

Мы можем предположить, что фильм A больше похож на фильм B, чем на фильм C, так как фильмы A и B имеют два общих тега (фантастика, школа), в то время как фильмы A и C не имеют ни одного общего тега.

Если бы мы рассматривали вычисление этого индекса для фильмов А и В, то получили бы $\frac{2}{4}$, так как в пересечении два тега, а в объединении — четыре. Для фильмов А и C мы получили бы $0$, так как пересечение множеств их тегов является пустым. Таким образом, показатель близости для фильмов А и В получился бы больше, и это подтвердило бы наши предположения об их большей схожести.

### КОСИНУСНАЯ БЛИЗОСТЬ

Подход с использованием индекса Жаккара помог нам создать интуитивное представление о том, что означает сходство набора категориальных значений. Подход с **косинусным сходством** немного сложнее и применяется для оценки близости массивов с числами. Он требует, чтобы мы представляли объекты в виде вектора.
Например, мы можем представить те же самые фильмы как набор из трёх вещественных чисел:

* фильм $A = (1.1, 2.3, 5.1)$;
* фильм $B = (1.3, 2.1, 4.9)$;
* фильм $C = (5.1, 6.2, 1.1)$.

Глядя на эти векторы, кажется, что фильмы А и В похожи друг на друга больше, чем, например, фильмы А и С, так как координаты фильмов А и В очень близки и различаются гораздо меньше, чем координаты фильмов А и С.

Чтобы вычислить косинусную близость, нам понадобится следующая формула:

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/19ae75390814099a1aaf8a660888b0d6/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/DST_MATH_ML_15_2_5.png)

По сути, мы ищем угол между векторами. Если два вектора совпадут (т. е. будут максимально близкими), то угол между ними будет равен нулю, а значит, косинус будет равен $1$. Если векторы будут направлены в противоположные стороны, косинус будет равен $-1$. Таким образом, мы можем получить для любых двух векторов значение в пределах от $1$ до $-1$ включительно, по которому можно определить, насколько векторы близки друг к другу.
Давайте вычислим косинусную близость, чтобы оценить, насколько наши догадки о близости фильмов соответствуют реальности.

Косинусная близость для фильмов А и B:

$\operatorname{sim}(A, B)=\frac{1.1\cdot 1.3+2.3 \cdot 2.1+5 \cdot 1^* 4.9}{\sqrt{1.1^2+2 .3^2+5.1^2} \sqrt{1.3^2+2.1^2+4.9^2}} \approx 0,999$

### Задание 2.1

Вычислите косинусную близость между векторами А и С. Результат округлите до трёх знаков после точки-разделителя.

In [1]:
A = [1.1, 2.3, 5.1]
C = [5.1, 6.2, 1.1]

sim = (A[0]*C[0] + A[1]*C[1] + A[2]*C[2])/ \
    ((A[0]**2 + A[1]**2 + A[2]**2)**0.5 * (C[0]**2+C[1]**2+C[2]**2)**0.5)

round(sim, 3)

0.551

Как нам известно, чем выше значение косинусной близости, тем больше сходство между векторами. Таким образом, наши изначальные предположения совпали с реальностью: векторы А и В действительно схожи между собой сильнее, чем векторы А и С. Значит, если человек поставил высокую оценку фильму А, то мы должны порекомендовать ему фильм В.

Итак, чтобы построить рекомендательную систему на основе контента, необходимо:

1. Для каждого продукта создать характеризующие его признаки.
2. Найти показатель близости между всеми продуктами.
3. Порекомендовать пользователю продукты, которые показывают наибольшую близость с теми продуктами, которые он высоко оценил.

Давайте реализуем подобную рекомендательную систему на практике. Будем работать с [датасетом](https://lms-cdn.skillfactory.ru/assets/courseware/v1/747dae7bf99b18ce3b24bd34aa7bc29b/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/netflix_titles.zip), содержащим информацию об оценивании фильмов на платформе Netflix.

**Признаки в данных:**

* `show_id` — id фильма,
* `type` — его тип (фильм или сериал),
* `title` — название,
* `director` — режиссер,
* `cast` — актерский состав,
* `country` — страна,
* `date_added` — дата добавления,
* `release_year` — год выхода на экраны,
* `rating` — рейтинг,
* `duration` — продолжительность,
* `listened_in` — жанр(-ы),
* `description` — описание.

В первую очередь нам необходимо определить, на основании чего мы будем рассматривать близость фильмов. Выберем для этой задачи описание фильма, ведь в нём, скорее всего, содержится много информации. Однако описание — это текст. Есть много подходов к преобразованию текста в вектор, и мы будем использовать подход **TF-IDF** (Term Frequency-Inverse Document Frequency).

> **Показатель TD-IDF** — это индикатор того, насколько релевантно слово в контексте документа.

Его можно определить следующим образом:

$$\text{TF-IDF(слова) = TF(слова) * IDF (слова)}$$

где:

* $\text{TF слова} = \frac{\text{Количество раз, когда слово встретилось в тексте}}{\text{Количество всех слов в тексте}}$

* $\text{IDF слова} = log \left (\frac{\text{Общее кол-во документов}}{\text{Кол-во документов, в которых встречается слово}}\right )$

Этот показатель возрастает пропорционально количеству раз, когда слово встречается в тексте, и уменьшается пропорционально количеству слов во всех текстах в целом.

Таким образом:

* Коэффициент будет выше, если слово характерно именно для этого текста, то есть встречается в данном тексте часто, но не встречается в других текстах.
* Коэффициент будет ниже, если слово не встречается почти нигде или встречается одинаковое количество раз во всех текстах, то есть не характеризует никакой текст в отдельности.

Если вам интересно подробнее изучить алгоритм создания такого представления, рекомендуем прочитать [статью](https://medium.com/analytics-vidhya/tf-idf-term-frequency-technique-easiest-explanation-for-text-classification-in-nlp-with-code-8ca3912e58c3).

Чтобы преобразовать текст по этому принципу, нам понадобится соответствующая функция из библиотеки `sklearn` — импортируем её:

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer


Далее учтём стоп-слова, т. е. предлоги и другие служебные части речи, которые не несут содержательной информации, и с учётом этого определим нашу модель:

In [3]:
model = TfidfVectorizer(stop_words='english')


Заполним пропуски пустыми строками:



In [4]:
import pandas as pd

df = pd.read_csv('https://lms-cdn.skillfactory.ru/assets/courseware/v1/747dae7bf99b18ce3b24bd34aa7bc29b/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/netflix_titles.zip')

df['description'] = df['description'].fillna('')

Трансформируем наши описания в матрицу:



In [5]:
feature_matrix = model.fit_transform(df['description'])


### Задание 2.2

Сколько столбцов в получившейся матрице?

In [6]:
feature_matrix

<7787x17905 sparse matrix of type '<class 'numpy.float64'>'
	with 107187 stored elements in Compressed Sparse Row format>

Теперь необходимо вычислить косинусную близость. Можно сделать это так:

In [7]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(feature_matrix, feature_matrix)

**Обратите внимание!** Мы используем здесь `linear_kernel()`, а не [`cosine_similarity()`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html#sklearn.metrics.pairwise.cosine_similarity), так как в косинусном расстоянии в знаменателе реализуется нормировка векторов, а TF-IDF создаёт уже нормализованные векторы.

Вернём индексацию и уберём дубликаты из данных:



In [8]:
indices = pd.Series(df.index,index=df['title']).drop_duplicates()


Теперь пропишем функцию для создания рекомендаций:



In [9]:
def get_recommendations(title):
    idx = indices[title]
    #вычисляем попарные коэффициенты косинусной близости
    scores = list(enumerate(cosine_sim[idx]))
    #сортируем фильмы на основании коэффициентов косинусной близости по убыванию
    scores = sorted(scores, key=lambda x: x[1], reverse=True)
    #выбираем десять наибольших значений косинусной близости; нулевую не берём, т. к. это тот же фильм
    scores =   scores[1:11]
    #забираем индексы
    ind_movie = [i[0] for i in scores]
    #возвращаем названия по индексам
    return df['title'].iloc[ind_movie]

Например, если мы хотим найти рекомендации по фильму "Star Trek", то функция будет выдавать следующий результат:

In [10]:
get_recommendations('Star Trek')

5788             Star Trek: The Next Generation
5787                      Star Trek: Enterprise
5786                 Star Trek: Deep Space Nine
5557                     She's Out of My League
134                                  7 Days Out
6664                        The Midnight Gospel
6023                                     Teresa
4863    Pinkfong & Baby Shark's Space Adventure
5104                                       Rats
5970                             Tales by Light
Name: title, dtype: object

### Задание 2.3

Найдите вторую рекомендацию для детского фильма "Balto", вышедшего на экраны в 1995 году:

In [11]:
get_recommendations('Balto').iloc[1]

'Vroomiz'

Итак, мы смогли создать рекомендации на основе контента и разобрались с принципом работы этого алгоритма. Теперь давайте посмотрим на преимущества и недостатки данного подхода.

**ПРЕИМУЩЕСТВА**

* **Для создания рекомендаций не требуются данные от других пользователей.** Как только пользователь выполнил поиск, просмотрел несколько продуктов и/или совершил несколько покупок, система фильтрации на основе контента может начать создавать соответствующие рекомендации. Это делает её идеальной для компаний и сервисов, у которых нет огромного количества пользователей для формирования выборки.
* **Рекомендации получаются очень релевантными для пользователя.** Рекомендации на основе контента могут быть в значительной степени адаптированы к интересам пользователя, включая рекомендации по нишевым товарам, поскольку метод основан на сопоставлении характеристик или атрибутов объекта базы данных с интересами пользователя.
* **Рекомендации прозрачны для пользователя.** Высокорелевантные рекомендации создают ощущение понятности алгоритмов для пользователя, повышая уровень его доверия к предлагаемым рекомендациям.
* **Вы избегаете проблемы «холодного старта».** Хотя фильтрация на основе контента требует первоначального ввода данных от пользователей, чтобы начать давать рекомендации, качество ранних рекомендаций обычно намного выше, чем у других подходов.
* **Системы фильтрации на основе содержания обычно проще в создании.** Основная работа заключается в создании характеристик, на основании которых будет вычисляться близость.

**НЕДОСТАТКИ**

* **Отсутствие новизны и разнообразия.** Вполне возможно, что человеку, который любит боевики, могли бы понравиться и фильмы ужасов. Однако мы не сможем ему их порекомендовать, если он сам не отметит подобные фильмы как понравившиеся.
* **Присвоенные характеристики могут быть неверными.** Рекомендации на основе контента хороши настолько, насколько хороши атрибуты (характеристики), при

Итак, мы разобрались с первым персонализированным методом для построения рекомендательных систем. В следующих юнитах мы детально изучим другие алгоритмы.

# 3. Коллаборативная фильтрация

✍ Следующий подход к построению рекомендательных систем, который мы изучим, — это коллаборативная фильтрация. Она основана на поиске сходства между пользователями или между продуктами. Используя этот метод, мы можем прогнозировать рейтинги на основе оценок похожих пользователей или похожих продуктов.

![](https://img.genial.ly/5fdc5ca1853b5759f6e69400/8d75cb1e-148b-4ba2-ba3c-aefac1e8a161.png)

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/0641f061288f15ee47c7986bf6c9c0fe/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/DST_MATH_ML_15_3_1.png)

Однако для начала давайте рассмотрим очень часто встречающуюся в рекомендательных системах концепцию — матрицу предпочтений.

Чтобы её получить, расположим в матрице клиентов по строкам, а продукты — по столбцам. На пересечении строк и столбцов разместим оценки, поставленные клиентами соответствующим продуктам: первый клиент поставил второму товару 3, третий клиент поставил первому товару 2 и так далее.

/|ТОВАР 1|ТОВАР 2|ТОВАР 3|ТОВАР 4|ТОВАР 5
-|-|-|-|-|-
КЛИЕНТ 1||3||5|
КЛИЕНТ 2|1||1|1|
КЛИЕНТ 3|2|||3|2
КЛИЕНТ 4||4|||5
КЛИЕНТ 5|5||2|3|4

На основе этих данных мы можем разделить пользователей на кластеры. Чтобы это сделать, можно взять некоторую меру близости для пользователей по их истории оценок и на основе полученных значений объединить пользователей в кластеры таким образом, чтобы похожие пользователи оказались в одной группе, а сильно отличающиеся — в разных. В таком случае оценку пользователя для продукта можно прогнозировать как среднюю оценку пользователей этого кластера, оценивших этот продукт.

Таким образом, если нам необходимо предсказать, как конкретный пользователь оценил фильм, мы анализируем оценки, поставленные данному фильму пользователями, которые принадлежат к тому же кластеру, что и изучаемый пользователь, и усредняем эти оценки. Так получается предсказание оценки фильма для нашего пользователя.

В целом, такой подход можно применять, однако у него есть ряд существенных недостатков:

* Нечего рекомендовать новым/нетипичным пользователям. Если появляется пользователь, который ни на кого не похож, мы не знаем, к какому кластеру его отнести. На начальных стадиях мы определяем его в случайный кластер, и рекомендации в таком случае будут плохими.
* Не учитывается специфика каждого пользователя. По сути, мы выявляем некоторые паттерны поведения и предпочтений и для каждого паттерна выделяем свои рекомендации. Однако на самом деле даже пользователи из одного кластера немного отличаются друг от друга, поэтому возникают неточности.
* Если оценок нет, то среднее арифметическое невозможно вычислить. Если в кластере никто не оценивал объект, сделать предсказание не получится, так как для предсказания нужно вычислить среднее арифметическое для оценок.

## КОЛЛАБОРАТИВНАЯ ФИЛЬТРАЦИЯ НА ОСНОВЕ ПАМЯТИ (MEMORY-BASED)

Чтобы решить перечисленные выше проблемы, обратимся к коллаборативной фильтрации, а точнее к memory-based-подходу, основанному на близости пользователей (user-based).

Напомним, что при memory-based-подходе хранится полная матрица взаимодействий (лайков, просмотров и т. д .) пользователя с продуктом.

### КОЛЛАБОРАТИВНАЯ ФИЛЬТРАЦИЯ НА ОСНОВЕ ПОЛЬЗОВАТЕЛЕЙ (USER-BASED-ПОДХОД)

> **Коллаборативная фильтрация на основе пользователей** — это метод, используемый для предсказания продуктов, которые могут понравиться пользователю, на основе оценок, выставленных этому продукту другими пользователями, имеющими схожие с целевым пользователем вкусы.

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/5c33a47165a42a7a6dd473ac39459bfd/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/DST_MATH_ML_15_3_2_ed.png)

В этом алгоритме мы заменяем жёсткую кластеризацию на следующую формулу и получаем предсказанную оценку пользователя $u$, которую он поставил элементу $i$:

$$\hat{r}_{u i}=\bar{r}_u+\frac{\sum_{v \in U_i} \operatorname{sim}(u, v)\left(r_{v i}-\bar{r}_v\right)}{\sum_{v \in U_i} \operatorname{sim}(u, v)}$$

Здесь используются следующие обозначения:

* $u$ и $v$ — индексы пользователей;
* $\bar{r}_u$ — средняя оценка пользователя ;
* $\bar{r}_v$ — средняя оценка пользователя ;
* $sim$ — функция схожести;
* $i$ — номер оцениваемого элемента.
Средняя оценка пользователя может быть никак не привязана к его интересам. По сути, это просто показатель того, как в среднем пользователь привык оценивать фильмы.

Оценка пользователя, которую мы предсказываем для него, состоит **из двух частей**:

* Непосредственно его средняя оценка.
* Слагаемое, состоящее из разницы в оценках с другими пользователями, т. е. похожести пользователей. Эта разница домножается на похожесть пользователей, то есть в числителе — средневзвешенная разница в оценках, а в знаменателе — сумма показателей схожести.

Каждому клиенту мы подбираем релевантный для него товар в рамках группы клиентов, но не решаем задачу кластеризации, а усредняем интересы данной группы в дистанции нескольких соседей. По сути, здесь мы руководствуемся идеей, что, например, видео можно порекомендовать человеку, если оно понравилось его друзьям.

Давайте разберём пример применения этого алгоритма.

В матрице четыре пользователя — Алиса, Рома, Катя и Женя. Они оценивают различные приложения из AppStore. Диапазон оценок — от 1 до 5. Знак '?' означает, что данный пользователь не оценил это приложение.

ИМЯ|APP1|APP2|APP3|APP4|APP5
-|-|-|-|-|-
Алиса|5|4|1|4|?
Рома|3|1|2|3|3
Катя|4|3|4|3|5
Женя|3|3|1|5|4

Вычислим сходство как коэффициент корреляции — такой подход также популярен.

Для начала найдём среднее значение рейтинга для каждого пользователя:

$$\bar{r}_i=\frac{\sum_p r_{i p}}{\sum p}$$

Таким образом получаем:

$\bar{r}_{\text {Алиса }}=3.5$

$\bar{r}_{Рома}=2.25$

$\bar{r}_{Катя}=3.5$

$\bar{r}_{Женя}=3$

Теперь вычисляем сходство между Алисой и всеми остальными пользователями:

$\operatorname{Sim}(Алиса, Рома)=\frac{((1.5 \cdot 0.75)+(0.5 \cdot (-1.25))+(-2.5 \cdot(-0.25))+(0.5 \cdot 0.75))}{\sqrt{\left(1.5^2+0.5^2+2.5^2+0.5^2\right)} \sqrt{\left(0.75^2+1.25^2+0.25^2+0.75^2\right)}}=0.301$

$\operatorname{Sim}(Алиса, Катя)=\frac{((1.5 \cdot 0.5)+(0.5 \cdot (-0.5))+(-2.5 \cdot 0.5)+(0.5 \cdot (-0.5))}{\sqrt{\left(1.5^2+0.5^2+2.5^2+0.5^2\right)} \sqrt{\left(0.5^2+0.5^2+0.5^2+0.5^2\right)}}=-0.33$

$\operatorname{Sim}(Алиса, Женя)=\frac{((1.5 \cdot 0)+(0.5 \cdot 0)+(-2.5 \cdot(-2))+(0.5 \cdot 2))}{\sqrt{\left(1.5^2+0.5^2+2.5^2+0.5^2\right)} \sqrt{\left(0^2+0^2+2^2+2^2\right)}}=0.707$

Теперь спрогнозируем рейтинг Алисы для приложения App5:

$r_{(\text {Алиса }, I 5)}=\bar{r}_{\text {Алиса }}+\frac{\left(\operatorname{sim}(\text { Алиса, } U 1) *\left(r_{U 1, I 5}-\bar{r}_{U 1}\right)\right)+\left(\operatorname{sim}(\text { Алиса,U2 }) *\left(r_{U 2, I 5}-\bar{r}_{U 2}\right)\right)+\left(\operatorname{sim}(Алиса, U 3) *\left(r_{U 3, I 5}-\bar{r}_{U 3}\right)\right.}{\operatorname{sim}(\text { Алиса, } U 1)+\operatorname{sim}(\text { Алиса,U2 })+\operatorname{sim}(Алиса, U 3)}$

$r_{(\text {Алиса, I5 })}=3.5+\frac{(0.301 * 0.75)+(-0.33 * 1.5)+(0.707 * 1)}{|0.301|+|-0.33|+|0.707|}=3.83$

Итак, мы смогли реализовать предсказание для user-based-подхода.



### КОЛЛАБОРАТИВНАЯ ФИЛЬТРАЦИЯ НА ОСНОВЕ ЭЛЕМЕНТОВ (ITEM-BASED-ПОДХОД)

Если мы транспонируем матрицу предпочтений и будем решать ту же самую задачу не для пользователей, а для объектов (items), то получим аналогичную задачу, которая является item-based-моделью коллаборативной фильтрации и даёт нам возможность предсказывать оценку следующим образом:

$$\hat{r}_{u i}=\bar{r}_i+\frac{\sum_{j \in I_u} \operatorname{sim}(i, j)\left(r_{u j}-\bar{r}_j\right)}{\sum_{j \in I_u} \operatorname{sim}(i, j)}$$

По формуле можно понять, что этот подход использует идею предыдущего, только теперь похожи не пользователи, а объекты. Возвращаясь к примеру с рекомендациями фильмов, теперь мы рекомендуем пользователю фильм, который похож на те фильмы, которые уже понравились этому пользователю ранее.

Кроме того, продуктов обычно больше, чем пользователей, поэтому векторы получатся большей размерности. Это даёт возможность получить более устойчивую модель с большей статистической значимостью.

Может показаться, что коллаборативная фильтрация в рамках item-based-подхода очень похожа на модель на основе контента. Однако это не так: item-based-модель рассматривает взаимодействия пользователей с продуктом, а content-based-модель — метаинформацию продукта.

Теперь давайте рассмотрим преимущества и недостатки коллаборативной фильтрации, основанной на памяти (memory-based):

**ПРЕИМУЩЕСТА**

* Может помочь пользователям обнаружить новые релевантные продукты из новых категорий, даже если пользователи не проявляют интерес к новым типам продуктов.
* Не требует подробных характеристик и контекстных данных о продуктах. По сути, для реализации нужна только матрица взаимодействий пользователей и продуктов.

**НЕДОСТАТКИ**

* Нехватка данных может привести к трудностям при рекомендации новых продуктов или при появлении новых пользователей, поскольку предложения основаны на исторических данных и взаимодействии. Таким образом, для этого подхода актуальна проблема холодного старта.
* По мере роста пользовательской базы алгоритмы страдают из-за большого объёма данных (очень ресурсоёмкие вычисления) и недостаточной масштабируемости.
* Отсутствие разнообразия в долгосрочной перспективе. Это может показаться нелогичным, поскольку смысл коллаборативной фильтрации заключается в том, чтобы рекомендовать пользователю новые товары. Однако поскольку алгоритмы функционируют на основе исторических рейтингов, они не будут рекомендовать товары с небольшим количеством оценок или ограниченным количеством данных. Популярные товары будут более популярны в долгосрочной перспективе, а новых и разнообразных вариантов будет не хватать.

Итак, мы рассмотрели memory-based-подходы коллаборативной фильтрации. Давайте перейдём к следующим подходам, которые относятся к категории model-based и основаны на разложениях матриц. После этого мы сможем сравнить изученные модели при решении практической задачи.

## MODEL-BASED-ПОДХОД

В подходе на основе моделей используются модели машинного обучения для прогнозирования и ранжирования взаимодействий между пользователями и элементами, с которыми они ещё не взаимодействовали. Эти модели обучаются на основе информации, уже имеющейся в матрице взаимодействий, с помощью различных алгоритмов, например матричной факторизации.

**Матричная факторизация** используется для генерации **латентных признаков** путём разложения **разрежённой** матрицы взаимодействия пользователя и продукта на две меньшие и **плотные** матрицы особенностей пользователей и продуктов.

- Под латентными признаками понимаются некоторые выделенные на основе данных факторы. Например, это может быть фактор приверженности определённому жанру фильмов.

- Мы получаем разрежённую матрицу из-за того, что продуктов очень много, и, очевидно, каждый пользователь взаимодействовал только с небольшим их количеством. Поэтому в матрице много пустых ячеек (нулей).

- Матрицы плотные, так как нам известны характеристики пользователей и продуктов.

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/d02212cd93a334020c696d2b7835e6eb/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/DST_MATH_ML_15_3_3.png)

Слева на иллюстрации мы видим разрежённую матрицу (X), а справа — матрицу пользователей U с их характеристиками (размерности k) и матрицу товаров V с их характеристиками (размерности k).

Рассмотрим два варианта матричной факторизации — SVD и ALS.

SVD — это сингулярное разложение, с которым вы уже подробно знакомились в [модуле по кластеризации](https://lms.skillfactory.ru/courses/course-v1:SkillFactory+DST-3.0+28FEB2021/jump_to_id/de3e2256746440e1a97c14a54697424f). Давайте вспомним суть SVD.

Любую прямоугольную матрицу A размера (n, m) можно представить в виде произведения трёх матриц:

В этой формуле:

* $U$ — матрица размера $n,n$. Все её столбцы ортогональны друг другу и имеют единичную длину. Такие матрицы называются **ортогональными**. Эта матрица содержит нормированные собственные векторы матрицы $AA^T$.
* $D$ — матрица размера $n,m$. На её главной диагонали стоят числа, называемые **сингулярными** числами (они являются корнями из собственных значений матриц $AA^T$ и $A^TA$), а вне главной диагонали стоят нули. Если мы решаем задачу снижения размерности, то элементы этой матрицы, если их возвести в квадрат, можно интерпретировать как дисперсию, которую объясняет каждая компонента.
* $V$ — матрица размера $m,m$. Она тоже **ортогональная** и содержит нормированные собственные векторы матрицы $A^TA$).

Данное разложение используют для того, чтобы представить матрицу предпочтений как разложение на матрицу с характеристиками пользователей и матрицу с характеристиками продуктов. Матрица U представляет связь между пользователями и латентными факторами, D — диагональная матрица, описывающая силу каждого латентного фактора, а V — матрица, описывающая связь между продуктами и латентными факторами.

Однако SVD — не единственная возможность для разложения матрицы. Второй популярный алгоритм — ALS (Alternating Least Square).

ALS — итеративный алгоритм разложения матрицы предпочтений на произведение двух матриц.

Чтобы понять лучше суть этого алгоритма, вспомним одну из известных нам функций потерь — RMSE.

Предположим, что есть m пользователей и n продуктов. Тогда у нас будут следующие матрицы:

* $R$ размерности $m*n$;
* $U$ размерности $m*k$;
* $P$ размерности $n*k$, где $k$ — количество латентных факторов.

Тогда функция потерь, которую мы будем минимизировать, будет следующей:

$$\begin{aligned} \text { loss } & =\min (y-\hat{y})^2 \\ & =\min \left(R-U * P^T\right)^2 \\ & =\min \sum_{m, n}\left(R_{m, n}-U_m * P_n^T\right)^2\end{aligned}$$

Здесь:

* $R$ — истинные показатели взаимодействия пользователя и продукта;
* $U*P^T$ — прогнозируемые показатели взаимодействия пользователя и продукта.

Далее мы, по сути, решаем метод наименьших квадратов для разницы матриц, на каждом шаге уменьшая ошибку сначала по факторам пользователей, а затем — по факторам товаров. Чтобы избежать ситуации переобучения, к ошибке добавляются регуляризационные коэффициенты.

ALS — это итерационный процесс оптимизации, в котором мы на каждой итерации пытаемся приблизиться к факторизованному представлению исходных данных.

Давайте рассмотрим реализацию ALS на «игрушечном» примере. Допустим, у нас есть некоторая матрица, которая содержит информацию про пользователей и их отношение к фильмам:

/|ФИЛЬМ 1|ФИЛЬМ 2|ФИЛЬМ 3
-|-|-|-
Пользователь 1|0.5|?|4
Пользователь 2|1|3|5

Однако нам неизвестно, какую оценку поставил Пользователь 1 Фильму 2 — её мы и будем пытаться предсказать с помощью ALS.

В соответствии с методом ALS мы хотим получить следующее разложение:

$\left(\begin{array}{ccc}0.5 & ? & 4 \\ 1 & 3 & 5\end{array}\right)=\left(\begin{array}{l}u_1 \\ u_2\end{array}\right)\left(\begin{array}{lll}p_1 & p_2 & p_3\end{array}\right)$

Для начала фиксируем матрицу U (вектор-столбец). Для неё можно выбрать любые случайные числа, но для удобства вычислений примем оба значения за единицу:

$\left(\begin{array}{ccc}0.5 & ? & 4 \\ 1 & 3 & 5\end{array}\right)=\left(\begin{array}{l}1 \\ 1\end{array}\right)\left(\begin{array}{lll}p_1 & p_2 & p_3\end{array}\right)$

Если мы перемножим матрицы в правой части равенства, то получим пять уравнений, в которых участвуют компоненты из вектора p:

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/7cfd0f260ead1de4a75aa302d7108876/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/DST_MATH_ML_15_3_eq4.png)

Поскольку существует единственное уравнение, определяющее вторую компоненту P, мы задаём её равной 3. Нам необходимо выбрать оставшиеся две компоненты так, чтобы средняя квадратичная ошибка была минимальной. Таким образом, вычисляем:

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/279fe917219154ac3ca65f2fb52443c0/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/DST_MATH_ML_15_3_eq5.png)

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/8ca933222d3532e8cc93cc2f8b5d79ae/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/DST_MATH_ML_15_3_eq6.png)

Разумеется, мы умеем вычислять точки минимума для функции, используя производные. Находим точки минимума для обоих случаев и получаем первую оценку для матрицы:

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/7ed8812fc95a1f755772a90238182c97/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/DST_MATH_ML_15_3_eq7.png)

Теперь мы оставляем компоненты матрицы P фиксированными и оптимизируем матрицу U. Это аналогично даёт нам следующие уравнения для U:

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/099c4c6f2311990b9a03599c4e81f1f6/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/DST_MATH_ML_15_3_eq8.png)

Далее мы можем минимизировать среднеквадратичную ошибку для компонентов матрицы U, как мы это делали ранее для компонентов матрицы P.

Повторяя эти итерации, мы в какой-то момент сходимся к оптимальным матрицам U и P. В данном примере после 20 итераций можно определить, что U и P даны в виде:

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/f6ef822ea53016f0a6d20afb87d1d906/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/DST_MATH_ML_15_3_eq9.png)

Теперь из этих матриц мы можем получить восстановленную матрицу прогнозируемых взаимодействий пользователей и продуктов, на основе которой мы можем составлять рекомендации.

## ПРАКТИКА

Мы рассмотрели несколько вариантов коллаборативной фильтрации на простейших примерах, и теперь пришло время практики с настоящими данными. Сначала мы будем использовать подход memory-based в модификации item-based, а затем SVD. В результате применения обоих алгоритмов мы сможем сравнить получившееся качество.

Для создания алгоритмов рекомендательной системы будем использовать библиотеку surprise.

Установим её:

In [12]:
!pip install scikit-surprise

**Примечание**. Заметьте, что в названии библиотеки содержится название уже известного нам модуля scikit — это не просто так: названия функций и аргументов, да и сам принцип работы этих модулей похожи.

В нашей задаче мы будем использовать датасет [movielens](https://lms-cdn.skillfactory.ru/assets/courseware/v1/6e47046882bad158b0efbb84cd5cb987/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/u.data.txt), который содержит информацию о фильмах и выставленных рейтингах с сайта [https://movielens.org/](https://movielens.org/).

Импортируем необходимые нам компоненты и считаем данные с помощью специального метода Reader:

In [13]:
import surprise
from surprise import Dataset
from surprise import Reader
from surprise.dataset import BUILTIN_DATASETS #с помощью данного объекта мы можем использовать встроенные датасеты
import pandas as pd


In [14]:
data = Dataset.load_builtin(
    name = 'ml-100k'
)

Чтобы обучать рекомендательные системы с помощью surprise, мы создали объект Dataset. Объект surprise.dataset — это набор данных, который содержит следующие поля в указанном порядке:

* идентификаторы пользователей,
* идентификаторы элементов,
* соответствующая оценка.

Преобразуем данные к формату pandas DataFrame для удобной работы с ними:

In [15]:
df = pd.DataFrame(data.raw_ratings, columns=['userId', 'movieId', 'rating', 'timestamp'])
df.head()

,userId,movieId,rating,timestamp
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596


В данных присутствуют следующие признаки:

* `userId` — идентификаторы пользователей сайта movielens;
* `movieId` — идентификаторы фильмов;
* `rating` — оценки фильмов, выставленные пользователями по шкале от 1 до 5;
* `timestamp` — время оценки фильма пользователем. Данный формат представления времени показывает, сколько секунд прошло с 1 января 1970 года.

### Задание 3.1

Сколько уникальных фильмов в наборе данных?

In [16]:
df['movieId'].nunique()

# 1682

1682

### Задание 3.2

Сколько уникальных пользователей в наборе данных?

In [17]:
df['userId'].nunique()

# 943

943

### Задание 3.3

Какая оценка встречается в наборе данных чаще всего? Введите ответ в виде целого числа.

In [18]:
df['rating'].value_counts()

# 4    34174
# 3    27145
# 5    21201
# 2    11370
# 1     6110
# Name: rating, dtype: int64

4.0    34174
3.0    27145
5.0    21201
2.0    11370
1.0     6110
Name: rating, dtype: int64

Библиотека surprise очень похожа на библиотеку sklearn, и тоже позволяет разбить данные на обучающую и тестовую выборки всего одной функцией — [surprise.model_selection.train_test_split()](https://surprise.readthedocs.io/en/stable/getting_started.html#train-test-split-and-the-fit-method).

### Задание 3.4

Разбейте данные на обучающую и тестовую выборки. Объём тестовой выборки должен составлять 25 % от общего объёма данных. В качестве значения параметра random_state возьмите число 13.

Сколько объектов попало в тестовую выборку?

In [19]:
trainset, testset = surprise.model_selection.train_test_split(
    data,
    test_size=0.25,
    random_state=13)

len(testset)

# 25000

25000

Импортируем функции для построения рекомендательных систем (SVD — для model-based-подхода и KNNBasic — для memory-basic-подхода) и для оценки качества результата.

In [20]:
from surprise import SVD, KNNBasic, accuracy

Теперь реализуем обычную коллаборативную фильтрацию. Выберем оценку схожести через косинусную близость и item-based-подход:

In [21]:
sim_options = {
    'name': 'cosine',
    'user_based': False
}

knn = KNNBasic(sim_options=sim_options)

Обучим алгоритм:

In [22]:
knn.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


Теперь давайте посмотрим, какие рекомендации мы получили, с помощью следующей программы:

In [23]:
predictions = knn.test(testset)
predictions_df = pd.DataFrame(predictions)
predictions_df.head()

,uid,iid,r_ui,est,details
0,7,633,5.0,4.199452,"{'actual_k': 40, 'was_impossible': False}"
1,422,287,3.0,3.470344,"{'actual_k': 40, 'was_impossible': False}"
2,804,163,3.0,3.571674,"{'actual_k': 40, 'was_impossible': False}"
3,189,480,5.0,4.222826,"{'actual_k': 40, 'was_impossible': False}"
4,238,546,3.0,3.473417,"{'actual_k': 17, 'was_impossible': False}"


После этого можно вывести результат, записанный в переменную predictions.

Информация о каждой паре будет содержать следующие характеристики:

* `uid` — id пользователя;
* `iid` — id элемента;
* `r_ui` (float) — реальный рейтинг, который этот пользователь поставил этому элементу;
* `est` (float) — предсказанный рейтинг.

### Задание 3.5

1. Каков реальный рейтинг, выставленный пользователем с ID 500 для фильма с ID 699?

2. Каков прогнозируемый рейтинг для пользователя с ID 500 и фильма с ID 699? Ответ округлите до двух знаков после точки-разделителя.

In [24]:
predictions_df[(predictions_df['uid']=='500') & (predictions_df['iid']=='699')]

,uid,iid,r_ui,est,details
946,500,699,3.0,3.47479,"{'actual_k': 40, 'was_impossible': False}"


Теперь необходимо вычислить RMSE для получившихся предсказаний:

In [25]:
accuracy.rmse(predictions)

# RMSE: 1.0272
# 1.0271678039029761

RMSE: 1.0272


1.0271678039029761

Если округлить результат до сотых, получаем $1.03$.

Итак, мы построили систему рекомендаций и даже оценили её качество. Но как же вывести рекомендации для конкретного пользователя?
Для начала давайте оформим наши предсказания в таблицу и отсортируем их по прогнозируемому рейтингу:

In [26]:
pred = pd.DataFrame(predictions)
pred.sort_values(by=['est'],inplace=True,ascending = False)

Теперь мы можем вывести рекомендуемые для конкретного пользователя фильмы, начиная от наиболее релевантного (с точки зрения рекомендаций) и заканчивая наименее релевантным.

In [27]:
recom = pred[pred.uid =='849']['iid'].to_list()
recom

# ['234', '427', '568', '174']

['234', '427', '568', '174']

### Задание 3.6

Реализуйте user-based-алгоритм. Какое значение RMSE получилось для коллаборативной фильтрации типа user-based? Ответ округлите до двух знаков после точки-разделителя.

In [28]:
sim_options = {
    'name': 'cosine',
    'user_based': True
}

knn = KNNBasic(sim_options=sim_options)

knn.fit(trainset)

predictions = knn.test(testset)

accuracy.rmse(predictions)

# RMSE: 1.0175
# 1.0174852296380237

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.0175


1.0174852296380237

### Задание 3.7

Теперь давайте сравним полученные результаты с результатами SVD-алгоритма. Реализуйте SVD с параметрами по умолчанию.

Какое значение RMSE получилось для SVD? Ответ округлите до двух знаков после точки-разделителя.

In [29]:
svd = SVD()

svd.fit(trainset)

predictions = svd.test(testset)

accuracy.rmse(predictions)

# RMSE: 0.9416
# 0.9416396832536824

RMSE: 0.9425


0.9425263348553188

У SVD наилучшее качество, так как значение ошибки наименьшее.

В этом юните мы не только разобрались с принципом работы различных вариаций коллаборативной фильтрации, но и реализовали их на практике, а также сравнили результаты, полученные с помощью разных подходов. В следующем юните вы познакомитесь с ещё одним подходом к построению рекомендательных систем, а также изучите ещё одну библиотеку, с помощью которой можно строить рекомендации.

# 4. Гибридные модели

✍  В предыдущих юнитах мы рассмотрели отдельные модели для построения рекомендательных систем. Иногда эти модели используются вместе — такой способ построения рекомендательной системы называется **гибридным**.

![](https://img.genial.ly/5fdc5ca1853b5759f6e69400/3c95a584-7f39-4a42-b05e-b3e8ea656de2.png)

> **Гибридная РС** — это особый тип рекомендательной системы, который представляет собой комбинацию из нескольких методов. Обычно это комбинация контентного подхода и коллаборативной фильтрации. Такое сочетание может помочь преодолеть недостатки, с которыми мы сталкиваемся при использовании этих методов по отдельности, а также в некоторых случаях может быть более эффективным.

Гибридные подходы к рекомендательным системам можно реализовать по-разному, например используя комбинацию из рекомендаций, полученных обеими системами, или используя взвешенную сумму предсказанных рейтингов.

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/0d19555fccdf48343e741b517e72bf21/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/DST_MATH_ML_15_4_1.png)

Давайте на практике рассмотрим, как создать рекомендательную систему с помощью гибридного подхода.

Разумеется, можно комбинировать различные подходы самостоятельно, однако для удобства уже реализован **модуль LightFM** — установим библиотеку через следующую команду:

In [30]:
!pip install lightfm

**Примечание**. К сожалению, при локальном использовании данной библиотеки очень часто возникают ошибки. Если вы столкнулись с этим, рекомендуем использовать Google Colab.

Импортируем нужные нам функции из этой библиотеки. На этом этапе сразу же загрузим инструменты оценки модели:

In [31]:
from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import precision_at_k, recall_at_k

Работать мы будем с датасетом [goodreads_book](https://lms-cdn.skillfactory.ru/assets/courseware/v1/c977535583bf2f85a2d15617e672d8f4/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/Gooddreadbooks.zip).

Goodreads — это сайт, на котором люди могут добавлять книги в каталоги, искать их, изучать аннотации и отзывы. Пользователи также могут создавать сообщества, в которых они рекомендуют друг другу различную литературу, ведут блоги и устраивают обсуждения.

Подгрузим все файлы, относящиеся к этому набору данных:

In [32]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [33]:
ratings = pd.read_csv('/content/drive/MyDrive/SkillFactory/Gooddreadbooks/ratings.csv') # Поставленные оценки
books = pd.read_csv('/content/drive/MyDrive/SkillFactory/Gooddreadbooks/books.csv') # Информация о книгах
tags = pd.read_csv('/content/drive/MyDrive/SkillFactory/Gooddreadbooks/tags.csv') # Информация о тегах
book_tags = pd.read_csv('/content/drive/MyDrive/SkillFactory/Gooddreadbooks/book_tags.csv') # Книги с тегами

Сначала посмотрим на набор данных books: в этих данных есть обычный id книги, а есть id книги в системе Goodreads — этот id отображён в признаке goodreads_book_id. В других данных (book_tags) указан только id книги в системе Goodreads, поэтому нам необходимо добавить туда обычный id.

### Задание 4.1

Добавьте в набор данных book_tags признак с обычным id книги, используя соответствие обычного id и id в системе Goodreads.

Какой обычный id у книги, которая имеет id 5 в системе Goodreads?

In [34]:
book_tags = book_tags.merge(
    books[['book_id','goodreads_book_id']],
    on = 'goodreads_book_id'
    )

book_tags[book_tags['goodreads_book_id'] == 5].iloc[0]['book_id']

# 18

18

### Задание 4.2

Далее нам необходимо оставить в наборе данных book_tags только те записи, теги для которых есть в данных tags.

Отфильтруйте данные таким образом, чтобы в наборе данных book_tags остались только те строки, в которых находятся теги, информация о которых есть в наборе данных tags.

Сколько объектов осталось?

In [35]:
book_tags = book_tags.merge(
    tags['tag_id'],
    on='tag_id')

book_tags.shape[0]

# 300738

300738

Отлично, мы подготовили информацию о тегах книг — это будет метаинформацией для построения рекомендательной системы. Теперь нам необходимо подготовить данные о взаимодействии пользователей и книг. Для этого нам понадобится файл ratings.

Оба набора данных (и про взаимодействия, и про метаинформацию) необходимо преобразовать в разрежённые матрицы. Это можно сделать с помощью специальной функции из модуля scipy:

In [36]:
!pip install scipy
from scipy.sparse import csr_matrix

Нам важно преобразовать данные в специальный формат, в котором хранятся разрежённые матрицы — будем использовать формат Compressed Sparse Row (CSR), подразумевающий подсчёт кумулятивной суммы количества элементов в строке вместо индексов строк.

Выглядит результат так:

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/dada594391a323bd95b0e7db9c136470/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/DST_MATH_ML_15_4_2.png)

Здесь хранится информация о том, сколько суммарно ненулевых элементов в данной строке и выше, индексы столбцов с ненулевыми значениями, сами значения и размерность матрицы.

* В первой строке обозначено, сколько накоплено ненулевых значений (в первой строке — суммарно 1, после второй строки — суммарно 4, после третьей строки — суммарно 4, после четвёртой строки — суммарно 6).
* Во второй строке показано, в каком столбце находится ненулевое значение.
* В третьей строке указаны сами значения.

Осуществляем преобразование следующим образом:

In [37]:
ratings_matrix = csr_matrix((
    ratings.rating,(
        ratings.user_id,ratings.book_id)))
# Передаём в качестве аргументов в функцию выставленный рейтинг (это будут
# значения матрицы), а также id пользователя и id книги (это будут индексы для
# строк и столбцов матрицы)

Теперь нам необходимо составить матрицу с метаданными. В качестве индексов будут выступать id книги и id тега, и если у этой книги есть рассматриваемый тег, то на пересечении соответствующих строки и столбца будет выставлена единица.

In [38]:
meta_matrix  = csr_matrix(([1]*len(book_tags),
                           (book_tags.book_id,book_tags.tag_id)))

### Задание 4.4

Давайте проверим, что всё получилось правильно.

Каково среднее арифметическое значений разрежённой матрицы с рейтингами? Ответ округлите до трёх знаков после точки-разделителя.

In [39]:
ratings_matrix.mean()

# 0.007086188900997592

0.007086188900997592

Отлично, данные подготовлены — теперь настало время определить модель, которую мы будем использовать. Сделаем это следующим образом:

In [40]:
model = LightFM(
    loss='warp-kos', # Определяем функцию потерь
    random_state=42, # Фиксируем случайное разбиение
    learning_rate=0.05, # Темп обучения
    no_components=100 # Размерность вектора для представления данных в модели
)

В качестве функции потерь мы выбрали значение `warp`, хотя, разумеется, это не единственный вариант. В модуле LightFM представлены следующие функции потерь:

* `logistic` — логистическая функция. Полезна в случаях, когда есть как положительные, так и отрицательные взаимодействия, например 1 и -1.
* `bpr` — байесовский персонализированный рейтинг. Можно применять, когда присутствуют только положительные взаимодействия.
* `warp` — парный взвешенный приблизительный ранг. Используется, если необходимо повысить качество именно в верхней части списка рекомендаций.
* `warp-kos` — модификация warp.

Разобьём данные на обучающую и тестовую выборки:

In [41]:
train, test = random_train_test_split(
    ratings_matrix, # Общая выборка
    test_percentage=0.2, # Размер тестовой выборки
    random_state=42 # Генератор случайных чисел
)

Теперь обучим модель на наших данных о взаимодействии, также используя метаданные о книгах. Для этого воспользуемся методом fit(). В этот метод передадим обучающую выборку, признаки товаров — item_features, количество эпох обучения (сколько раз мы будем показывать модели исходный датасет, чтобы она лучше выучила данные) — epochs, а также параметр verbose для отслеживания процесса обучения:

In [42]:
model = model.fit(
    train, # Обучающая выборка
    item_features=meta_matrix, # Признаки товаров
    epochs=10, # Количество эпох
    verbose=True # Отображение обучения
)

Epoch: 100%|██████████| 10/10 [11:55<00:00, 71.59s/it]


**Обратите внимание:** из-за трудоёмкости вычислений обучение модели и оценка качества могут занимать вплоть до 15-20 минут (зависит от мощности компьютера). Не волнуйтесь, это нормальная ситуация.

**Примечание.** Если вы работаете через Google Collab, то для того, чтобы ускорить процесс обучения, вы можете в метод fit() передать параметр num_threads, в котором необходимо указать количество используемых потоков процессора. Задайте его значение, например, равным 6.

### Задание 4.5

Оцените качество полученной модели с помощью функции precision_at_k, передав в неё три аргумента: модель, тестовые данные и обозначение метаданных (item_features = meta_matrix).

Примечание. Процесс расчёта метрик рекомендательной системы также является довольно затратным по времени. Для ускорения этого процесса вы можете передать параметр num_threads, чтобы указать количество потоков процессора, используемых для вычислений.

Выведите среднее арифметическое и округлите его до двух знаков после точки-разделителя.

In [43]:
precision_at_k(
    model,
    test,
    item_features = meta_matrix
).mean()

0.023226766

В рекомендательных системах метрики интерпретируются иначе, чем в задачах классификации. Показатели точности РС считаются хорошими, если они находятся в районе 0.1-0.3.

У нас получился не слишком высокий, но довольно неплохой результат. Чтобы его улучшить, можно попробовать следующее:

* Поработать над предобработкой данных, добавив в них дополнительную информацию о товарах. Также можно попробовать воспользоваться иным способом создания разреженной матрицы, например, форматом coo_matrix() или csc_matrix(), которые также входят в библиотеку scipy. Подробнее почитать о них вы можете здесь.
* Поиграться с параметрами модели LightFM — поуправлять темпом обучения (learning_rate), размерностью вектора для представления (no_components), количеством эпох обучения (epochs) и функцией потерь (loss).

**Примечание.** Для предсказания рейтинга нового пользователя можно воспользоваться методом predict():

```py
scores = model.predict(<индекс интересующего пользователя>, np.arange(n_items), user_features=new_user_feature)
```

Подробнее об этом можно узнать [здесь](https://making.lyst.com/lightfm/docs/lightfm.html#lightfm.LightFM.predict).

Итак, мы реализовали гибридную РС и оценили её качество. Какие преимущества и недостатки есть у данного подхода?

ПРЕИМУЩЕСТВА

* Использование разных моделей позволяет компенсировать их недостатки и использовать преимущества каждой модели для составления рекомендаций.
* Благодаря использованию сразу нескольких методов такой подход позволяет получить более персонализированные и точные рекомендации.

НЕДОСТАТКИ

* Такие модели обычно имеют высокую вычислительную сложность и из-за этого долго обучаются.
* Для обучения таких моделей требуются большие объёмы данных и поступление новых данных для обновления рекомендаций.

В этом юните мы обсудили, что такое гибридная рекомендательная система, и познакомились с библиотекой LightFM, с помощью которой можно обучать подобные системы. В следующем юните вас ожидает последний в этом модуле подход к построению рекомендательных систем — искусственные нейронные сети.

# 5. Современные методы: глубокое обучение

✍ В этом юните мы познакомимся с последним подходом к построению рекомендательных систем — использование глубокого обучения.

**Глубокое обучение (Deep Learning, DL)** — это современное и эффективное решение для многих задач машинного обучения, таких как компьютерное зрение или обработка естественного языка. Deep Learning во многих случаях превосходит классические методы, которые мы рассматривали ранее. Поэтому в последнее время глубокое обучение всё чаще применяется и в рекомендательных системах. Многие крупные компании, такие как AirBnB, Google, Home Depot, LinkedIn и Pinterest, используют рекомендательные системы, построенные именно на основе глубокого обучения.

Преимущества использования нейронных сетей:

* **Как правило, DL-модели дают более высокое качество.** Стандартные ML-модели проигрывают глубокому обучению, особенно в ситуациях с большим объёмом данных.
* **DL-модели обладают большей гибкостью.** В рамках одной модели вы можете получить ответы на такие вопросы, как «Добавит ли пользователь товар в корзину?», «Начнёт ли он оформление заказа с этим товаром?» или «Купит ли он этот товар?».
* **Можно включать в модель данные совершенно разных типов**, в т. ч. текстовые данные (используя на них все инструменты NLP) или изображения (используя свёрточные нейронные сети).

Существует множество архитектур нейронных сетей, которые можно использовать для разработки рекомендательных систем. Сейчас мы рассмотрим простейшую архитектуру, чтобы в целом разобраться с принципом создания РС с использованием глубокого обучения.

Прежде чем погрузиться в архитектуру модели, давайте познакомимся с концепцией эмбеддингов.

> **Эмбеддинг** — это пространство низкой размерности, которое отражает взаимосвязь векторов из пространства более высокой размерности.

Чтобы лучше понять эту концепцию, давайте рассмотрим пример создания эмбеддингов пользователей.

Представим, что у нас есть различные данные о пользователях, которые мы хотим преобразовать в векторы с двумя координатами, отражающие всего две характеристики:

* степень симпатии к триллерам;
* степень симпатии к мелодрамам.

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/1437ee64e45332efce38b7b953b2beeb/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/DST_MATH_ML_15_5_1.png)

Теперь давайте представим некоторого пользователя Алису. Алиса предпочитает смотреть триллеры и не очень любит мелодрамы. Если в зависимости от её действий и оценок фильмов мы сможем оценить её любовь к триллерам и нелюбовь к мелодрамам в численном эквиваленте, то мы получим вектор, характеризующий предпочтения Алисы в плане жанров:

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/1619680c35e15a23928665ebd0eb5c44/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/DST_MATH_ML_15_5_2.png)

Рассмотрим ещё одного пользователя — Машу. Маша в целом является киноманом, так что примерно одинаково любит и триллеры, и мелодрамы. По аналогии с Алисой попробуем создать вектор, который характеризует предпочтения Маши.

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/5bdfbd4cc82a8b8169c2973428062c5a/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/DST_MATH_ML_15_5_3.png)


Создав такие векторы-представления для всех пользователей, мы получим полный набор эмбеддингов.

По сути, эмбеддинги уменьшают размерность данных, оставляя осмысленность в их отображении. При таком преобразовании пользователи, предпочтения которых похожи, находятся рядом в плоскости (или пространстве), а пользователи, предпочтения которых отличаются, находятся далеко друг от друга.

Конечно, необязательно использовать для формирования эмбеддинга только два числа — для вектора можно взять любое количество компонент. Причём чем больше будет измерений, тем выше будет точность представления всех особенностей пользователя и, как следствие, выше будет точность модели. Однако следует понимать, что такое увеличение точности происходит за счёт **роста сложности** модели и **увеличения количества времени**, необходимого для её обучения.

Ровно таким же образом мы можем создать эмбеддинги и для фильмов, отобразив информацию о них в пространство меньшей размерности.

Теперь, когда мы разобрались с эмбеддингами, можно переходить к самой архитектуре нейронной сети. Рассмотрим следующий пример архитектуры:

![](https://lms-cdn.skillfactory.ru/assets/courseware/v1/2f8ee4a823c8e68c6459ace461da87a5/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/DST_MATH_ML_15_5_4.png)

В качестве входных данных для нейронной сети мы передаём вектор характеристик пользователя и вектор для фильма (Inputs на схеме выше).

Из обоих этих векторов получаются эмбеддинги, о которых мы поговорили ранее (обозначены как Embedded User Vector и Embedded Item Vector). Далее эти эмбеддинги проходят через несколько полносвязных слоёв, на выходе из которых они преобразуются в вектор-предсказание. Например, на схеме выше видно, что вероятность первого класса (показывает, что элемент нерелевантен) — 0.2, а второго (показывает, что элемент релевантен) — 0.8. Следовательно, мы делаем выбор в пользу второго и рекомендуем этот продукт пользователю. Собственно, по такому алгоритму и обучается эта нейронная сеть.

Конечно, наше знакомство с глубоким обучением получилось очень поверхностным, но сейчас для вас важно просто понимать его принцип.

Давайте разберём несложную задачу, при решении которой мы обучим настоящую нейронную сеть и используем её для создания рекомендаций.

Мы будем использовать модуль tensorflow, в котором реализовано много полезных методов для имплементации (внедрения) нейронных сетей. Установим его:

In [44]:
!pip install tensorflow

Для начала импортируем из него функции, которые понадобятся нам для решения задачи:

In [45]:
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from tensorflow.keras.models import Model

Мы будем использовать данные из предыдущего юнита, но лишь те, которые содержат информацию об оценках, выставленных книгам пользователями. Загрузим данные:

In [46]:
df = ratings

### Задание 5.1

Разбейте данные на обучающую и тестовую выборки в отношении 4:1. В качестве значения параметра random_state возьмите число 42.

Сколько объектов теперь находится в обучающей выборке?

In [47]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(
    df,
    test_size=0.2,
    random_state=42)


train.shape[0]

785404

### Задание 5.2

Запишите количество уникальных книг в переменную n_books.

Сколько в наборе данных уникальных книг?

In [48]:
n_books = df['book_id'].nunique()
n_books

10000

### Задание 5.2

Запишите количество уникальных книг в переменную n_books.

Сколько в наборе данных уникальных книг?

In [49]:
n_users = df['user_id'].nunique()
n_users

53424

В первую очередь нам необходимо создать эмбеддинги для книг и пользователей. Создаём эмбеддинги для книг:

In [50]:
book_input = Input(shape=[1], name="Book-Input")
book_embedding = Embedding(n_books+1, 5, name="Book-Embedding")(book_input)
book_vec = Flatten(name="Flatten-Books")(book_embedding)

Сначала мы задаём размерность входного слоя. После этого определяем размер эмбеддинга — в данном случае снижаем размерность до 5. Далее мы разворачиваем результат в массив с одним измерением с помощью слоя Flatten().

Делаем то же самое для пользователей:

In [51]:
user_input = Input(shape=[1], name="User-Input")
user_embedding = Embedding(n_users+1, 5, name="User-Embedding")(user_input)
user_vec = Flatten(name="Flatten-Users")(user_embedding)

Теперь, когда мы создали представления как для книг, так и для пользователей, нам необходимо соединить их:

In [52]:
conc = Concatenate()([book_vec, user_vec])


Далее начинаем «собирать» нашу нейронную сеть из слоёв. Dense обозначает полносвязный слой. Также мы обозначаем для него количество нейронов и данные, которые идут на вход.

In [53]:
fc1 = Dense(128, activation='relu')(conc)
fc2 = Dense(32, activation='relu')(fc1)
out = Dense(1)(fc2)

Собираем модель — передаём входные данные для книг и пользователей, а также архитектуру нейронной сети:

In [54]:
model2 = Model([user_input, book_input], out)


Также нам необходимо задать алгоритм оптимизации и метрику, которую мы будем оптимизировать. В данном случае будем использовать метод adam и хорошо известную вам среднеквадратичную ошибку:

In [55]:
model2.compile(optimizer = 'adam',loss =  'mean_squared_error')


Теперь будем обучать нашу модель:



In [56]:
history = model2.fit([train.user_id, train.book_id], train.rating, epochs=5, verbose=1)

Epoch 1/5
24544/24544 [==============================] - 171s 7ms/step - loss: 0.7994
Epoch 2/5
24544/24544 [==============================] - 151s 6ms/step - loss: 0.6885
Epoch 3/5
24544/24544 [==============================] - 155s 6ms/step - loss: 0.6610
Epoch 4/5
24544/24544 [==============================] - 158s 6ms/step - loss: 0.6377
Epoch 5/5
24544/24544 [==============================] - 156s 6ms/step - loss: 0.6158


В параметр эпох передаём значение 5: у нас будет реализовано пять эпох — пять обучений нейронной сети. На каждой из эпох обновляются веса для минимизации ошибки.

Теперь можно оценить качество:

In [57]:
model2.evaluate([test.user_id, test.book_id], test.rating)


6136/6136 [==============================] - 10s 2ms/step - loss: 0.7059


0.7058582305908203

**Примечание.** К сожалению, результаты этого алгоритма нельзя зафиксировать стандартным ramdom_state, к которому мы привыкли: применяемые методы не используют такой параметр. Поэтому мы опустим здесь сравнение результатов, однако посмотрим, как можно настроить нейронную сеть.

Обычно для улучшения качества модели каким-то образом модифицируют нейронную сеть: дополняют её, увеличивают время обучения. Добавим ещё один полносвязный слой с восемью нейронами после полносвязного слоя с 32 нейронами. Обучим нейронную сеть, реализовав десять эпох:

In [58]:
fc1 = Dense(128, activation='relu')(conc)
fc2 = Dense(32, activation='relu')(fc1)
fc3 = Dense(8, activation='relu')(fc2)
out = Dense(1)(fc3)

model2 = Model([user_input, book_input], out)
model2.compile('adam', 'mean_squared_error')
result = model2.fit([train.user_id, train.book_id], train.rating, epochs=10, verbose=1)
model2.evaluate([test.user_id, test.book_id], test.rating)

Epoch 1/10
24544/24544 [==============================] - 157s 6ms/step - loss: 0.6254
Epoch 2/10
24544/24544 [==============================] - 167s 7ms/step - loss: 0.5817
Epoch 3/10
24544/24544 [==============================] - 167s 7ms/step - loss: 0.5623
Epoch 4/10
24544/24544 [==============================] - 158s 6ms/step - loss: 0.5463
Epoch 5/10
24544/24544 [==============================] - 159s 6ms/step - loss: 0.5332
Epoch 6/10
24544/24544 [==============================] - 162s 7ms/step - loss: 0.5217
Epoch 7/10
24544/24544 [==============================] - 161s 7ms/step - loss: 0.5116
Epoch 8/10
24544/24544 [==============================] - 160s 7ms/step - loss: 0.5025
Epoch 9/10
24544/24544 [==============================] - 157s 6ms/step - loss: 0.4949
Epoch 10/10
6136/6136 [==============================] - 12s 2ms/step - loss: 0.7755


0.7755383253097534

Качество получившейся модели не будет выше качества предыдущей, так как усложнение сети или увеличение количества эпох не всегда даёт высокое качество. Здесь главное, что вы научились корректировать архитектуру нейронной сети.

Теперь вы познакомились со всеми методами построения рекомендательных систем — самое время перейти к последнему, практическому юниту.

# 6. Практика

✍ Итак, вы познакомились с основными методами построения рекомендательных систем, и теперь настало время закрепить полученные знания на практике. В предыдущем модуле мы начали строить РС для сервиса чтения статей CI&T DeskDrop. В этом юните мы продолжим работу над ней.

**Примечание.** Если у вас не сохранился код, который мы использовали ранее, вы можете найти его в [ноутбуке](https://lms-cdn.skillfactory.ru/assets/courseware/v1/2ac7ffe70755c69c7d88642ed0ea990c/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/RecSys_unit6.ipynb).



In [59]:
articles_df = pd.read_csv('https://lms-cdn.skillfactory.ru/assets/courseware/v1/9f0e8eb4ddd03415fdd4db4a89a2b0d3/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/shared_articles.zip')


In [60]:
interactions_df = pd.read_csv('https://lms-cdn.skillfactory.ru/assets/courseware/v1/186647c8bd3fdb43b78fbc84ace97aed/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/users_interactions.zip')

interactions_df.personId = interactions_df.personId.astype(str)
interactions_df.contentId = interactions_df.contentId.astype(str)
articles_df.contentId = articles_df.contentId.astype(str)

In [61]:
event_type = {
   'VIEW': 1.0,
   'LIKE': 2.0,
   'BOOKMARK': 2.5,
   'FOLLOW': 3.0,
   'COMMENT CREATED': 4.0,
}

In [62]:
interactions_df['eventWeight'] = interactions_df['eventType'].apply(
    lambda x: event_type[x])

In [63]:
users_interactions_count_df = (
    interactions_df
    .groupby(['personId', 'contentId'])
    .first()
    .reset_index()
    .groupby('personId').size())

users_with_enough_interactions_df = \
    users_interactions_count_df[users_interactions_count_df >= 5].reset_index()[['personId']]

In [64]:
import numpy as np

interactions_from_selected_users_df = interactions_df.loc[np.in1d(interactions_df.personId,
            users_with_enough_interactions_df)]

In [65]:
import math

In [66]:
def smooth_user_preference(x):
    return math.log(1+x, 2)

In [67]:
interactions_from_selected_users_df.head(1)

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry,eventWeight
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN,1.0


In [68]:
interactions_full_df = (
    interactions_from_selected_users_df
    .groupby(['personId', 'contentId']).eventWeight.sum()
    .apply(smooth_user_preference)
    .reset_index().set_index(['personId', 'contentId'])
)
interactions_full_df['last_timestamp'] = (
    interactions_from_selected_users_df
    .groupby(['personId', 'contentId'])['timestamp'].last()
)

interactions_full_df = interactions_full_df.reset_index()

In [69]:
split_ts = 1475519545
interactions_train_df = interactions_full_df.loc[interactions_full_df.last_timestamp < split_ts].copy()
interactions_test_df = interactions_full_df.loc[interactions_full_df.last_timestamp >= split_ts].copy()

Так как теперь вам известны более сложные алгоритмы построения рекомендательных систем, начнём с них.

Для начала необходимо построить матрицу, в которой по столбцам будут находиться id статей, по строкам — id пользователей, а на пересечениях строк и столбцов — оценка взаимодействия пользователя со статьёй. Если взаимодействия не было, в соответствующей ячейке должен стоять ноль.

In [70]:
ratings = pd.pivot_table(
    interactions_train_df,
    values='eventWeight',
    index="personId",
    columns="contentId",
).fillna(0)


### Задание 6.1

Найдите оценку взаимодействия пользователя с ID -1032019229384696495 со статьёй с ID 943818026930898372. Результат округлите до двух знаков после точки-разделителя.

Примечание. Здесь и далее (пока не будет указано иное) необходимо работать с обучающей выборкой.

In [71]:
round(ratings.loc["-1032019229384696495", "943818026930898372"], 2)

2.32

Теперь давайте попробуем применить memory-based-подход коллаборативной фильтрации.

Примечание. Данных достаточно много, поэтому для увеличения скорости работы преобразуйте таблицу в массив numpy.

In [72]:
ratings_m = ratings.values

### Задание 6.2

Найдите среднее арифметическое всех чисел в получившемся массиве. Результат округлите до трёх знаков после точки-разделителя.

In [73]:
ratings_m.mean()

0.016673743043640697

Перейдём к реализации коллаборативной фильтрации. Ранее мы делали это с помощью библиотеки surprise, однако это не всегда удобно, так как эта библиотека имеет ограниченное количество метрик для оценки качества и небольшой потенциал для более тонкой настройки алгоритма. Поэтому давайте попробуем реализовать алгоритмы коллаборативной фильтрации «с нуля». Такая практика применяется, если необходимо выстроить более сложную систему, чем могут предложить готовые модули. Кроме того, «ручная» реализация алгоритмов позволит лучше понять принцип их работы.

### Задание 6.3

Постройте матрицу схожести. Для этого вычислите все попарные коэффициенты корреляции для матрицы, полученной в предыдущем задании. Для каждой пары учитывайте только ненулевые значения (так как нулевые обозначают отсутствие взаимодействия и не интересуют нас). Выведите результат, полученный в ячейке с третьим индексом по строкам и сороковым — по столбцам.

In [74]:
similarity_users = np.zeros((len(ratings_m), len(ratings_m)))
for i in (range(len(ratings_m)-1)):
    for j in range(i+1, len(ratings_m)):

        mask_uv = (ratings_m[i] != 0) & (ratings_m[j] != 0)
        ratings_v = ratings_m[i, mask_uv]
        ratings_u = ratings_m[j, mask_uv]

        similarity_users[i,j] = np.corrcoef(ratings_v, ratings_u)[0, 1]
        similarity_users[j,i] = similarity_users[i,j]

similarity_users[3,40]

/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:495: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:181: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2821: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2680: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2680: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2829: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2830: RuntimeWarning:

-0.3333333333333333

Теперь у нас есть матрицы схожести пользователей. Их можно использовать для построения рекомендаций. Чтобы это сделать, надо реализовать следующий алгоритм.

**Для каждого пользователя:**

1. Найти пользователей с похожестью больше 0.

2. Для каждой статьи вычислить долю пользователей (среди выделенных на первом шаге), которые взаимодействовали со статьёй.

3. Порекомендовать статьи (не более 10) с наибольшими долями со второго шага (среди тех, которые пользователь ещё не видел).


In [75]:
interactions = (
    interactions_train_df
    .groupby('personId')['contentId'].agg(lambda x: list(x))
    .reset_index()
    .rename(columns={'contentId': 'true_train'})
    .set_index('personId')
)

interactions['true_test'] = (
    interactions_test_df
    .groupby('personId')['contentId'].agg(lambda x: list(x))
)

interactions['true_test'] = [ [] if x is np.NaN else x for x in interactions['true_test'] ]

prediction_user_based = []

for i in range(len(similarity_users)):
    users_sim = similarity_users[i] > 0
    if not any(users_sim):
        prediction_user_based.append([])
    else:
        tmp_recommend = np.argsort(ratings_m[users_sim].sum(axis=0))[::-1]
        tmp_recommend = ratings.columns[tmp_recommend]
        recommend = np.array(tmp_recommend)[~np.in1d(tmp_recommend, interactions.iloc[i]["true_train"])][:10]
        prediction_user_based.append(list(recommend))

interactions['prediction_user_based'] = prediction_user_based




### Задание 6.4

Постройте рекомендательную систему по алгоритму, описанному выше. Найдите первую рекомендацию для строки 35 (если считать с нуля).

In [76]:
prediction_user_based[35][0]

'-5148591903395022444'

После того как сделаны предсказания, можно вычислить качество по метрике, которую мы определили в предыдущем модуле при решении этой задачи:

In [77]:
def calc_precision(column):
    return ( interactions.apply(  lambda row:len(set(row['true_test']).intersection(
                set(row[column]))) /min(len(row['true_test']) + 0.001, 10.0), axis=1)).mean()

### Задание 6.5

Вычислите точность полученного предсказания. Ответ округлите до трёх знаков после точки-разделителя.

In [78]:
round(calc_precision('prediction_user_based'), 3)

0.005

### Задание 6.6

Теперь реализуем рекомендательную систему с использованием SVD.

Разложите матрицу взаимодействий пользователей со статьями с помощью функции svd из модуля scipy. Найдите максимальное значение в получившейся матрице U. Результат округлите до двух знаков после точки-разделителя.

In [79]:
from scipy.linalg import svd

In [80]:
U, sigma, V = svd(ratings)
U.max()

0.7071067811865459

Значения матрицы с сингулярными числами отсортированы по убыванию. Допустим, мы хотим оставить только первые 100 компонент и получить скрытые представления размерности 100. Для этого необходимо оставить 100 столбцов в матрице U, только первые 100 значений из sigma (и сделать из них диагональную матрицу) и 100 столбцов в матрице V. Затем необходимо перемножить преобразованные матрицы.

In [81]:
k = 100
s = np.diag(sigma[:k])
U = U[:, 0:k]
V = V[0:k, :]

round(s.sum(), 2)

2096.53

Теперь мы можем сделать предсказание по полученной матрице.

Примечание. Помните, что не нужно учитывать статьи, которые уже были просмотрены пользователем.

Найдите для каждого пользователя статьи с наибольшими оценками в восстановленной матрице.

### Задание 6.8

Вычислите качество полученного предсказания, используя всё ту же метрику точности. Ответ округлите до трёх знаков после точки-разделителя.

In [82]:
new_ratings = pd.DataFrame(
    U.dot(s).dot(V), index=ratings.index, columns=ratings.columns
)
top_k = 10
predictions = []

for personId in interactions.index:
    prediction = (
        new_ratings.loc[personId].sort_values(ascending=False).index.values
    )

    predictions.append(
        list(
            prediction[
                np.in1d(prediction, interactions.loc[personId, "true_train"], invert=True)
            ]
        )[:top_k]
    )

interactions["prediction_svd"] = predictions

calc_precision("prediction_svd")

0.012212989310270756

Итак, мы реализовали два алгоритма коллаборативной фильтрации буквально с нуля! Теперь для полноты картины давайте реализуем на этих данных гибридную модель и посмотрим, какое качество получится. Для этого воспользуемся уже изученной библиотекой LightFM.

### Задание 6.9

Возьмите матрицу, подготовленную в задании 6.1. Преобразуйте её в разреженную матрицу:

In [83]:
from scipy.sparse import csr_matrix
ratings_matrix = csr_matrix(ratings)
# передаём в качестве аргументов в функцию выставленный рейтинг
# (это будут значения матрицы), а также id пользователя и id книги (это будут
# индексы для строк и столбцов матрицы)

Воспользовавшись функцией `random_train_test_split()` из библиотеки `lightfm`, разделите данные на валидационную и обучающую выборки в соотношении 1:2 (30% на валидационную выборку, 70% на обучающую). В качестве значения параметра random_state возьмите число 13.

Обучите модель LightFM со 100 компонентами, параметром `random_state = 13`, темпом обучения 0.05 и функцией потерь `'warp'`. Обратите внимание на то, что так как в данном случае у нас нет item-признаков, то параметр item_features задавать не нужно.

Вычислите показатель точности (precision@k) при k = 10. Ответ округлите до двух знаков после точки-разделителя.

In [84]:
model = LightFM(loss='warp', #определяем функцию потерь
                random_state=13, #фиксируем случайное разбиение
                learning_rate=0.05, #темп обучения
                no_components=100) #размерность вектора для представления данных в модели

train,test = random_train_test_split(ratings_matrix, test_percentage=0.3, random_state=13)
model.fit(train)
prec_score = precision_at_k(model, test).mean()


print(round(prec_score, 2))

0.04


В данном случае модель «из коробки» показала наилучший результат, однако это совсем не показатель того, что стоит пользоваться исключительно готовыми функциями. Зная тонкости работы алгоритмов, вы можете создавать собственные гибридные системы, настраивать отдельные алгоритмы и добиваться ещё лучших результатов.

# 7. Итоги

⭐ Вы закончили изучение рекомендательных систем — поздравляем!

В этом модуле вы:

* разобрались с принципом работы всех основных моделей рекомендательных систем;
* научились реализовывать их на практике;
* смогли построить рекомендательную систему, использовав различные подходы.

Если вы хотите **подробнее изучить продвинутые алгоритмы построения РС**, рекомендуем обратить внимание на следующие источники:

* [Разбор конфигураций гибридных рекомендательных моделей.](https://medium.com/analytics-vidhya/7-types-of-hybrid-recommendation-system-3e4f78266ad8)
* [Обзорная статья об использовании глубокого обучения для построения рекомендательных систем.](https://arxiv.org/pdf/1707.07435.pdf)

[Скачайте](https://lms-cdn.skillfactory.ru/assets/courseware/v1/71ddf5dc39af7072937ce80011585bd9/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/DST_%D0%91%D0%BB%D0%BE%D0%BA_7_%D0%9C%D0%BE%D0%B4%D1%83%D0%BB%D1%8C_MATH_ML-15._%D0%A0%D0%B5%D0%BA%D0%BE%D0%BC%D0%B5%D0%BD%D0%B4%D0%B0%D1%82%D0%B5%D0%BB%D1%8C%D0%BD%D1%8B%D0%B5_%D1%81%D0%B8%D1%81%D1%82%D0%B5%D0%BC%D1%8B._%D0%A7%D0%B0%D1%81%D1%82%D1%8C_II.pdf) конспект к модулю ↓ — он поможет вам лучше структурировать материал и время от времени возвращаться к самым важным понятиям и принципам.